# Notebook Modules

In [1]:
%load_ext literary.module

In [2]:
import sys
from pathlib import Path
from ..hook import install_import_hook
from ..transpile.patch import patch

ImportError: attempted relative import with no known parent package

In [ ]:
def determine_package_name(path: Path) -> str:
    """Determine the corresponding importable name for a package directory given by
    a particular file path. Return `None` if path is not contained within `sys.path`.

    :param path: path to package
    :return:
    """
    for p in sys.path:
        # If the cwd is only on `sys.path` exactly, then it can't
        # be a submodule. However, let's first look for parent paths
        if str(path) == p:
            continue

        # Resolve path relative to `sys.path`
        try:
            relative_path = path.relative_to(p)
        except ValueError:
            continue

        return ".".join(relative_path.parts)

    # Nothing was found on `sys.path`, so we abort
    return None

In [ ]:
def load_ipython_extension(ipython):
    """Load the import hook and setup the global state for the Literary extension.
    When IPython invokes this function, the determined package root path will be
    added to `sys.path`.

    :param ipython: IPython shell instance
    """
    # Ensure that notebook hook is installed
    install_import_hook()
    
    cwd = Path.cwd()
    
    # Ensure CWD is not on sys.path
    sys.path = [p for p in sys.path if Path(p).resolve() != cwd]

    # Set `__package__` for consumer notebook
    ipython.user_ns.update(
        {
            "__package__": determine_package_name(cwd),
            "patch": patch,
        }
    )